# Reconocimiento de imágenes

Reconocimiento de fotografías de las caras de 'Ariel Sharon', 'Colin Powell', 'Donald Rumsfeld', 'George W Bush',       'Gerhard Schroeder', 'Hugo Chavez' y 'Tony Blair'.

El conjunto de datos utilizado en esta práctica es un extracto preprocesado de las *Labeled Faces in the Wild*, también conocido como LFW:

http://vis-www.cs.umass.edu/lfw/

Se puede descargar directamente de http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (Al pulsar el link, se descargan 233 MB de datos)

También se encuentra disponible entre los datasets que incorpora la librería ***Scikit Learn***, por lo que ***para esta práctica se descargarán fácilmente haciendo uso del cargador de datos de esta librería (fetch_lfw_people()***).

In [1]:
# Imports de librerías y módulos necesarios (completar)
import logging


In [ ]:
# Muestra del progreso de cada salida
logging.basicConfig(level = logging.INFO, format = '%(asctime)s %(message)s')

En primer lugar, vamos a descargar los datos, si aún no están en el disco, cargándolos como matrices numpy. El conjunto de datos extraído solo conservará imágenes de personas que tengan al menos 70 imágenes diferentes (lo que podemos especificar mediante *min_faces_per_person*) y se utilizará un ratio de 0.4 para cambiar el tamaño de la imagen de cada cara (*resize*).

In [2]:
# Descargar los datos y cargarlos como matrices numpy
# Almacenarlos en una variable llamada lfw_people


El resultado debe ser el siguiente diccionario:

In [7]:
lfw_people

{'data': array([[254.33333 , 254.      , 252.      , ...,  87.      ,  88.666664,
          87.      ],
        [ 39.      ,  50.666668,  47.      , ..., 117.666664, 115.      ,
         133.      ],
        [ 89.666664, 103.666664, 126.333336, ..., 175.33333 , 183.33333 ,
         182.66667 ],
        ...,
        [ 86.666664,  80.      ,  74.333336, ...,  44.333332,  50.      ,
          44.666668],
        [ 50.666668,  65.333336,  88.333336, ..., 196.66667 , 178.66667 ,
         165.66667 ],
        [ 30.      ,  27.      ,  33.      , ...,  35.      ,  35.666668,
          61.      ]], dtype=float32),
 'images': array([[[254.33333 , 254.      , 252.      , ...,  65.666664,
           51.333332,  40.333332],
         [253.33333 , 251.66667 , 247.66667 , ...,  66.666664,
           52.      ,  42.333332],
         [240.66667 , 231.33333 , 211.66667 , ...,  61.333332,
           49.      ,  41.666668],
         ...,
         [ 74.333336,  53.666668,  31.333334, ...,  97.333336,
     

In [3]:
# Mostrar tamaño de las imágenes (número de muestras, alto, ancho)
# Calcularlo a partir del valor "images" del diccionario lfw_people


In [4]:
# Almacenar las variables (es decir, las características, no el target)
# del conjunto de datos, a su vez, en una variable llamada X.
# Para ello, usar el valor "data" del diccionario lfw_people.


In [5]:
# Almacenar el número de variables del conjunto de datos, 
# a su vez, en una variable llamada n_features.
# Calcular el número de características a partir de X.


In [6]:
# Almacenar la etiqueta a predecir en una variable llamada y


In [7]:
# Almacenar los nombres asociados a los targets en una variable llamada target_names


In [8]:
# Almacenar el número de clases (calculado a partir de la variable anterior, target_names)
# en una variable llamada n_classes


Si todo ha ido bien, los resultados deben ser los siguientes:

In [13]:
print(f"Total dataset size:")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"n_classes: {n_classes}")

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


## Train test split

In [9]:
# Partición en subconjuntos de train (0.75%) y test (0.25%)
# Fijar semilla (random_state) en 42


Una vez divididos los datos en dos particiones, efectuar un análisis de componentes principales (*caras propias* o *eigencaras*) para reducir las dimensiones a 150.

## Reducción de dimensiones (PCA)

In [21]:
n_components = 150

print(f"Extracción de las top {n_components} eigenfaces a partir de {X_train.shape[0]} caras")

pca = PCA(n_components = n_components, svd_solver= 'randomized', whiten = True).fit(X_train)
eigenfaces = pca.components_.reshape((n_components, h, w))

Extracción de las top 150 eigenfaces a partir de 966 caras


In [22]:
# Proyectar los datos de entrada sobre la base ortonormal de las eigencaras 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

De ahora en adelante, utilizar X_train_pca para entrenar y X_test_pca para evaluar el modelo.

## Modelado

### Primera versión: Support Vector Classifier (SVC)

A continuación, entrenar un clasificador de tipo SVC (Support Vector Classifier) utilizando los datos proyectados sobre la base ortonormal de las eigencaras (variable X_train_pca).

Previamente, hacer una búsqueda de hiperparámetros de tipo *Grid* para este clasificador en el siguiente espacio de hiperparámetros:

- Valores del hiperparámetro C: 1e3, 5e3, 1e4, 5e4, 1e5.
- Valores del hiperparámetro gamma: 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1

Utilizar los siguientes hiperparámetros fijos para el clasificador:
- kernel de tipo 'rbf'
- class_weight de tipo 'balanced'

Mostrar el mejor estimador encontrado.

***Medir el tiempo que tarda la ejecución de la búsqueda de hiperparámetros, junto al entrenamiento (usar %%time al principio de la celda).***

In [ ]:
%%time


Una vez conseguido el mejor estimador para este espacio de hiperparámetros, evaluar el modelo en el subconjunto de test. Para ello:
- Calcular las predicciones para todo el subconjunto de test y almacenarlas en una variable llamada y_pred.
- Mostrar el informe de clasificación para este subconjunto (classification_report)
- Mostrar la matriz de confusión para este subconjunto (confusion_matrix)

In [11]:
# Evaluar los resultados en el subconjunto de test


In [25]:
# Mostrar las imágenes en forma de una galería de retratos
def plot_gallery(images, titles, h, w, n_row = 3, n_col = 4):
    
    """Función para crear una galería de retratos"""
    
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

# Componer los títulos, de forma que muestren la etiqueta predicha frente a la real
def title(y_pred, y_test, target_names, i):
    
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    
    return 'Predicted: %s\nReal:      %s' % (pred_name, true_name)

In [ ]:
prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

In [ ]:
# Galería de las eigencaras más significativas 

eigenface_titles = ["Eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

### Segunda versión: Multilayer Perceptron Classifier (MLP)

A continuación, entrenar un clasificador de tipo MLP (Multilayer Perceptron Classifier) utilizando los datos proyectados sobre la base ortonormal de las eigencaras (variable X_train_pca).

Previamente, hacer una búsqueda de hiperparámetros de tipo Grid para este clasificador en el siguiente espacio de hiperparámetros:

- Valores del hiperparámetro hidden_layer_sizes: (10), (10, 10), (10, 10, 10).
- Valores del hiperparámetro solver: sgd, adam

Utilizar los siguientes hiperparámetros fijos para el clasificador:

- Función de activación de tipo 'relu' para las capas ocultas
- Shuffle igual a True
- Máximo número de iteraciones igual a 5000

***Medir el tiempo que tarda la ejecución de la búsqueda de hiperparámetros, junto al entrenamiento (usar %%time al principio de la celda).***

In [ ]:
%%time


Una vez conseguido el mejor estimador para este espacio de hiperparámetros, evaluar el modelo en el subconjunto de test. Repetir el proceso efectuado para el SVC, mostrando también la galería de imágene de predicciones vs etiquetas reales.

In [14]:
# Evaluar los resultados en el subconjunto de test


## Conclusiones

¿Qué conclusiones podrías extraer tras haber entrenado estos dos clasificadores para este conjunto de imágenes? ¿Qué ocurre si bajamos el número de iteraciones máximas a 1000 para el MLP?